In [2]:
from bs4 import BeautifulSoup as BS
import numpy as np
import spacy
from Legacy.LexicalDiversity import get_ld_features, get_ld_features_dict, get_feature_matrix
from DataReader.XMLReader import XMLReader
from nltk.tokenize import sent_tokenize
from scipy.stats import pearsonr
from matplotlib import pyplot as plt


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: the LexicalDiversity module is deprecated. Use the Pipline components instead
  after removing the cwd from sys.path.


<h1>Lexical Variation + Lexical Sophistication Features</h1>

<h3>This part of the notebook extracts the feature_matrix and stores it, if you already have the feature matrices, do not run this but skip to the next cell.</h3>

In [9]:
#Level 1 data
level = "b1"

feature_matrix, grades, essays = get_feature_matrix("dataset_binned/level_" + level + ".xml", 1000)

#Save the feature matrix

1000
Extracting features for 1000 essays: [                                                  ---------------------------------------------------]


In [10]:
current = np.load("feature_matrices/level_b1_fm.npy")
current = current[:1000]
current == feature_matrix[:1000]

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [3]:
print(feature_matrix.shape)
copy = np.copy(feature_matrix)
print(copy.shape)
#feature_matrix = feature_matrix[:, 14:]

(1001, 28)
(1001, 28)


In [4]:
print(copy.shape)
np.save("level_" + level + "_fm.npy", copy)
np.save("level_" + level + "_grades.npy", grades)

(1001, 28)


<h3>Create the feature names array</h3>

In [ ]:
feature_names = ['LV_W', 'LV_WT', 'LV_WT1', 'LV_TTR', 'LV_CTTR', 'LV_RTTR', 'LV_HDD', 'LV_DUGA', 'LV_MAAS', 'LV_SUMM',
                         'LV_YULEK','LV_MTLD','LV_MSTTR','LV_MATTR']
feature_names += ["LS_FPC_NG", "LS_FPC_NA", "LS_FPC_TC", "LS_FPC_BS", "LS_FPC_CA", "LS_FPC_CT", "LS_FPC_CGA1",
            "LS_FPC_CGA2", "LS_FPC_CGA3", "LS_FOMN_NG", "LS_FOMN_NA", "LS_FOMN_TC", "LS_FOMN_BS", "LS_FOMN_CA"]



#print(feature_names)
#print(feature_matrix.shape)
#print(feature_matrix[0, :])
#print(grades.shape)

<h2>Level A1 (10 000 essays closest to the mean token count)</h2>

In [ ]:
feature_names = np.array(feature_names)

level="a1"

feature_matrix = np.load("level_" + level + "_fm.npy")
grades = np.load("level_" + level + "_grades.npy")

pearson_correlation_lv = []
p_scores_lv = []

pearson_correlation_ls = []
p_scores_ls = []

for column in feature_matrix[:, :14].T:
    calcs = pearsonr(column, grades)
    pearson_correlation_lv.append(calcs[0])
    p_scores_lv.append(calcs[1])

pearson_correlation_lv = np.array(pearson_correlation_lv)
    
top_ten_indices_lv = np.abs(pearson_correlation_lv).argsort()[-10:][::-1].astype("int32")
print("Top ten indicies: ", top_ten_indices_lv)


for column in feature_matrix[:, 14:].T:
    calcs = pearsonr(column, grades)
    pearson_correlation_ls.append(calcs[0])
    p_scores_ls.append(calcs[1])

pearson_correlation_ls = np.array(pearson_correlation_ls)
top_ten_indices_ls = np.abs(pearson_correlation_ls).argsort()[-10:][::-1].astype("int32")
print("Top ten indicies: ", top_ten_indices_ls)
colors = np.array(10)
#colors = ["blue" for element in feature_names[top_ten_indices] if element[:2] == "LV"]
#colors = ["green" for element in feature_names[top_ten_indices] if element[:2] == "LS"]


#print("Top ten values: " , pearson_correlation_lv[top_ten_indices_lv])

ordinal_values = np.arange(10)

plt.figure(figsize=(15, 10))
plt.bar(ordinal_values-0.2, pearson_correlation_lv[top_ten_indices_lv], color="blue", align="center", label="Lexical Variation", width=0.2)
plt.title("Feature to grade correlation")
plt.xlabel("Feature names")
plt.ylabel("Correlation to grade")
plt.xticks(ordinal_values-0.5, feature_names[top_ten_indices_lv])

for i, v in enumerate(pearson_correlation_lv[top_ten_indices_lv]):
    plt.text(ordinal_values[i] - 0.2, v + 0.001, str(round(v, 4)), rotation=45)

#plt.figure(figsize=(15, 10))
plt.bar(ordinal_values, pearson_correlation_ls[top_ten_indices_ls], color="green", align="center", label="Lexical Sophistication", width=0.2)
plt.title("Feature to grade correlation")
plt.xlabel("Feature names")
plt.ylabel("Correlation to grade")
plt.xticks(ordinal_values, zip(feature_names[top_ten_indices_lv], feature_names[top_ten_indices_ls+14]), rotation=45)

for i, v in enumerate(pearson_correlation_ls[top_ten_indices_ls]):
    plt.text(ordinal_values[i], v + 0.001, str(round(v, 4)), rotation=45)

plt.legend()
plt.show()



highest_value_index_lv = np.argmax(pearson_correlation_lv)

plt.figure(figsize=(15, 10))
plt.plot(feature_matrix[:, highest_value_index_lv], grades, "*")
plt.gray()
plt.title("Highest correlation feature with the grades("+ feature_names[highest_value_index_lv] + ")")
plt.xlabel(feature_names[highest_value_index_lv])
plt.ylabel("Grades")
plt.show()


highest_value_index_ls = np.argmax(pearson_correlation_ls)

plt.figure(figsize=(15, 10))
plt.plot(feature_matrix[:, highest_value_index_ls+14], grades, "*")
plt.gray()
plt.title("Highest correlation feature with the grades("+ feature_names[highest_value_index_ls + 14] + ")")
plt.xlabel(feature_names[highest_value_index_ls+14])
plt.ylabel("Grades")
plt.show()

<h2>Level A1: Lexical Sophistication feature table</h2>

In [ ]:
import pandas as pd

pearson_correlation_ls = np.array(pearson_correlation_ls)
p_scores_ls = np.array(p_scores_ls)

data = np.stack([feature_names[14:], pearson_correlation_ls, p_scores_ls])

df = pd.DataFrame(data.T, index=np.arange(14), columns=["Feature", "Pearson_Correlation", "p_score"])
df


<h2>Level A2 (10 000 essays closest to the mean token count)</h2>

In [ ]:
level="a2"

feature_matrix = np.load("level_" + level + "_fm.npy")
grades = np.load("level_" + level + "_grades.npy")

pearson_correlation_lv = []
p_scores_lv = []

pearson_correlation_ls = []
p_scores_ls = []

for column in feature_matrix[:, :14].T:
    calcs = pearsonr(column, grades)
    pearson_correlation_lv.append(calcs[0])
    p_scores_lv.append(calcs[1])

pearson_correlation_lv = np.array(pearson_correlation_lv)
    
top_ten_indices_lv = np.abs(pearson_correlation_lv).argsort()[-10:][::-1].astype("int32")
print("Top ten indicies: ", top_ten_indices_lv)


for column in feature_matrix[:, 14:].T:
    calcs = pearsonr(column, grades)
    pearson_correlation_ls.append(calcs[0])
    p_scores_ls.append(calcs[1])

pearson_correlation_ls = np.array(pearson_correlation_ls)
top_ten_indices_ls = np.abs(pearson_correlation_ls).argsort()[-10:][::-1].astype("int32")
print("Top ten indicies: ", top_ten_indices_ls)
colors = np.array(10)
#colors = ["blue" for element in feature_names[top_ten_indices] if element[:2] == "LV"]
#colors = ["green" for element in feature_names[top_ten_indices] if element[:2] == "LS"]


#print("Top ten values: " , pearson_correlation_lv[top_ten_indices_lv])

ordinal_values = np.arange(10)

plt.figure(figsize=(15, 10))
plt.bar(ordinal_values-0.2, pearson_correlation_lv[top_ten_indices_lv], color="blue", align="center", label="Lexical Variation", width=0.2)
plt.title("Feature to grade correlation")
plt.xlabel("Feature names")
plt.ylabel("Correlation to grade")
plt.xticks(ordinal_values-0.5, feature_names[top_ten_indices_lv])

for i, v in enumerate(pearson_correlation_lv[top_ten_indices_lv]):
    plt.text(ordinal_values[i] - 0.2, v + 0.001, str(round(v, 4)), rotation=45)

#plt.figure(figsize=(15, 10))
plt.bar(ordinal_values, pearson_correlation_ls[top_ten_indices_ls], color="green", align="center", label="Lexical Sophistication", width=0.2)
plt.title("Feature to grade correlation")
plt.xlabel("Feature names")
plt.ylabel("Correlation to grade")
plt.xticks(ordinal_values, zip(feature_names[top_ten_indices_lv], feature_names[top_ten_indices_ls+14]), rotation=45)

for i, v in enumerate(pearson_correlation_ls[top_ten_indices_ls]):
    plt.text(ordinal_values[i], v + 0.001, str(round(v, 4)), rotation=45)

plt.legend()
plt.show()



highest_value_index_lv = np.argmax(pearson_correlation_lv)

plt.figure(figsize=(15, 10))
plt.plot(feature_matrix[:, highest_value_index_lv], grades, "*")
plt.gray()
plt.title("Highest correlation feature with the grades("+ feature_names[highest_value_index_lv] + ")")
plt.xlabel(feature_names[highest_value_index_lv])
plt.ylabel("Grades")
plt.show()


highest_value_index_ls = np.argmax(pearson_correlation_ls)

plt.figure(figsize=(15, 10))
plt.plot(feature_matrix[:, highest_value_index_ls+14], grades, "*")
plt.gray()
plt.title("Highest correlation feature with the grades("+ feature_names[highest_value_index_ls + 14] + ")")
plt.xlabel(feature_names[highest_value_index_ls+14])
plt.ylabel("Grades")
plt.show()

<h2>Level A2: Lexical Sophistication feature table</h2>

In [ ]:
pearson_correlation_ls = np.array(pearson_correlation_ls)
p_scores_ls = np.array(p_scores_ls)

data = np.stack([feature_names[14:], pearson_correlation_ls, p_scores_ls])

df = pd.DataFrame(data.T, index=np.arange(14), columns=["Feature", "Pearson_Correlation", "p_score"])
df

<h2>Level B1 (10 000 essays closest to the mean token count)</h2>

In [ ]:
level="b1"

feature_matrix = np.load("level_" + level + "_fm.npy")
grades = np.load("level_" + level + "_grades.npy")

pearson_correlation_lv = []
p_scores_lv = []

pearson_correlation_ls = []
p_scores_ls = []

for column in feature_matrix[:, :14].T:
    calcs = pearsonr(column, grades)
    pearson_correlation_lv.append(calcs[0])
    p_scores_lv.append(calcs[1])

pearson_correlation_lv = np.array(pearson_correlation_lv)
    
top_ten_indices_lv = np.abs(pearson_correlation_lv).argsort()[-10:][::-1].astype("int32")
print("Top ten indicies: ", top_ten_indices_lv)


for column in feature_matrix[:, 14:].T:
    calcs = pearsonr(column, grades)
    pearson_correlation_ls.append(calcs[0])
    p_scores_ls.append(calcs[1])

pearson_correlation_ls = np.array(pearson_correlation_ls)
top_ten_indices_ls = np.abs(pearson_correlation_ls).argsort()[-10:][::-1].astype("int32")
print("Top ten indicies: ", top_ten_indices_ls)
colors = np.array(10)
#colors = ["blue" for element in feature_names[top_ten_indices] if element[:2] == "LV"]
#colors = ["green" for element in feature_names[top_ten_indices] if element[:2] == "LS"]


#print("Top ten values: " , pearson_correlation_lv[top_ten_indices_lv])

ordinal_values = np.arange(10)

plt.figure(figsize=(15, 10))
plt.bar(ordinal_values-0.2, pearson_correlation_lv[top_ten_indices_lv], color="blue", align="center", label="Lexical Variation", width=0.2)
plt.title("Feature to grade correlation")
plt.xlabel("Feature names")
plt.ylabel("Correlation to grade")
plt.xticks(ordinal_values-0.5, feature_names[top_ten_indices_lv])

for i, v in enumerate(pearson_correlation_lv[top_ten_indices_lv]):
    plt.text(ordinal_values[i] - 0.2, v + 0.001, str(round(v, 4)), rotation=45)

#plt.figure(figsize=(15, 10))
plt.bar(ordinal_values, pearson_correlation_ls[top_ten_indices_ls], color="green", align="center", label="Lexical Sophistication", width=0.2)
plt.title("Feature to grade correlation")
plt.xlabel("Feature names")
plt.ylabel("Correlation to grade")
plt.xticks(ordinal_values, zip(feature_names[top_ten_indices_lv], feature_names[top_ten_indices_ls+14]), rotation=45)

for i, v in enumerate(pearson_correlation_ls[top_ten_indices_ls]):
    plt.text(ordinal_values[i], v + 0.001, str(round(v, 4)), rotation=45)

plt.legend()
plt.show()



highest_value_index_lv = np.argmax(pearson_correlation_lv)

plt.figure(figsize=(15, 10))
plt.plot(feature_matrix[:, highest_value_index_lv], grades, "*")
plt.gray()
plt.title("Highest correlation feature with the grades("+ feature_names[highest_value_index_lv] + ")")
plt.xlabel(feature_names[highest_value_index_lv])
plt.ylabel("Grades")
plt.show()


highest_value_index_ls = np.argmax(pearson_correlation_ls)

plt.figure(figsize=(15, 10))
plt.plot(feature_matrix[:, highest_value_index_ls+14], grades, "*")
plt.gray()
plt.title("Highest correlation feature with the grades("+ feature_names[highest_value_index_ls + 14] + ")")
plt.xlabel(feature_names[highest_value_index_ls+14])
plt.ylabel("Grades")
plt.show()

<h2>Level B1: Lexical Sophistication feature table</h2>

In [ ]:
pearson_correlation_ls = np.array(pearson_correlation_ls)
p_scores_ls = np.array(p_scores_ls)

data = np.stack([feature_names[14:], pearson_correlation_ls, p_scores_ls])

df = pd.DataFrame(data.T, index=np.arange(14), columns=["Feature", "Pearson_Correlation", "p_score"])
df

<h2>Level B2 (10 000 essays closest to the mean token count)</h2>

In [ ]:
level="b2"

feature_matrix = np.load("level_" + level + "_fm.npy")
grades = np.load("level_" + level + "_grades.npy")

pearson_correlation_lv = []
p_scores_lv = []

pearson_correlation_ls = []
p_scores_ls = []

for column in feature_matrix[:, :14].T:
    calcs = pearsonr(column, grades)
    pearson_correlation_lv.append(calcs[0])
    p_scores_lv.append(calcs[1])

pearson_correlation_lv = np.array(pearson_correlation_lv)
    
top_ten_indices_lv = np.abs(pearson_correlation_lv).argsort()[-10:][::-1].astype("int32")
print("Top ten indicies: ", top_ten_indices_lv)


for column in feature_matrix[:, 14:].T:
    calcs = pearsonr(column, grades)
    pearson_correlation_ls.append(calcs[0])
    p_scores_ls.append(calcs[1])

pearson_correlation_ls = np.array(pearson_correlation_ls)
top_ten_indices_ls = np.abs(pearson_correlation_ls).argsort()[-10:][::-1].astype("int32")
print("Top ten indicies: ", top_ten_indices_ls)
colors = np.array(10)
#colors = ["blue" for element in feature_names[top_ten_indices] if element[:2] == "LV"]
#colors = ["green" for element in feature_names[top_ten_indices] if element[:2] == "LS"]


#print("Top ten values: " , pearson_correlation_lv[top_ten_indices_lv])

ordinal_values = np.arange(10)

plt.figure(figsize=(15, 10))
plt.bar(ordinal_values-0.2, pearson_correlation_lv[top_ten_indices_lv], color="blue", align="center", label="Lexical Variation", width=0.2)
plt.title("Feature to grade correlation")
plt.xlabel("Feature names")
plt.ylabel("Correlation to grade")
plt.xticks(ordinal_values-0.5, feature_names[top_ten_indices_lv])

for i, v in enumerate(pearson_correlation_lv[top_ten_indices_lv]):
    plt.text(ordinal_values[i] - 0.2, v + 0.001, str(round(v, 4)), rotation=45)

#plt.figure(figsize=(15, 10))
plt.bar(ordinal_values, pearson_correlation_ls[top_ten_indices_ls], color="green", align="center", label="Lexical Sophistication", width=0.2)
plt.title("Feature to grade correlation")
plt.xlabel("Feature names")
plt.ylabel("Correlation to grade")
plt.xticks(ordinal_values, zip(feature_names[top_ten_indices_lv], feature_names[top_ten_indices_ls+14]), rotation=45)

for i, v in enumerate(pearson_correlation_ls[top_ten_indices_ls]):
    plt.text(ordinal_values[i], v + 0.001, str(round(v, 4)), rotation=45)

plt.legend()
plt.show()



highest_value_index_lv = np.argmax(pearson_correlation_lv)

plt.figure(figsize=(15, 10))
plt.plot(feature_matrix[:, highest_value_index_lv], grades, "*")
plt.gray()
plt.title("Highest correlation feature with the grades("+ feature_names[highest_value_index_lv] + ")")
plt.xlabel(feature_names[highest_value_index_lv])
plt.ylabel("Grades")
plt.show()


highest_value_index_ls = np.argmax(pearson_correlation_ls)

plt.figure(figsize=(15, 10))
plt.plot(feature_matrix[:, highest_value_index_ls+14], grades, "*")
plt.gray()
plt.title("Highest correlation feature with the grades("+ feature_names[highest_value_index_ls + 14] + ")")
plt.xlabel(feature_names[highest_value_index_ls+14])
plt.ylabel("Grades")
plt.show()

<h2>Level B2: Lexical Sophistication feature table</h2>

In [ ]:
pearson_correlation_ls = np.array(pearson_correlation_ls)
p_scores_ls = np.array(p_scores_ls)

data = np.stack([feature_names[14:], pearson_correlation_ls, p_scores_ls])

df = pd.DataFrame(data.T, index=np.arange(14), columns=["Feature", "Pearson_Correlation", "p_score"])
df

<h2>Level C1 (10 000 essays closest to the mean token count)</h2>

In [ ]:
level="c1"

feature_matrix = np.load("level_" + level + "_fm.npy")
grades = np.load("level_" + level + "_grades.npy")

pearson_correlation_lv = []
p_scores_lv = []

pearson_correlation_ls = []
p_scores_ls = []

for column in feature_matrix[:, :14].T:
    calcs = pearsonr(column, grades)
    pearson_correlation_lv.append(calcs[0])
    p_scores_lv.append(calcs[1])

pearson_correlation_lv = np.array(pearson_correlation_lv)
    
top_ten_indices_lv = np.abs(pearson_correlation_lv).argsort()[-10:][::-1].astype("int32")
print("Top ten indicies: ", top_ten_indices_lv)


for column in feature_matrix[:, 14:].T:
    calcs = pearsonr(column, grades)
    pearson_correlation_ls.append(calcs[0])
    p_scores_ls.append(calcs[1])

pearson_correlation_ls = np.array(pearson_correlation_ls)
top_ten_indices_ls = np.abs(pearson_correlation_ls).argsort()[-10:][::-1].astype("int32")
print("Top ten indicies: ", top_ten_indices_ls)
colors = np.array(10)
#colors = ["blue" for element in feature_names[top_ten_indices] if element[:2] == "LV"]
#colors = ["green" for element in feature_names[top_ten_indices] if element[:2] == "LS"]


#print("Top ten values: " , pearson_correlation_lv[top_ten_indices_lv])

ordinal_values = np.arange(10)

plt.figure(figsize=(15, 10))
plt.bar(ordinal_values-0.2, pearson_correlation_lv[top_ten_indices_lv], color="blue", align="center", label="Lexical Variation", width=0.2)
plt.title("Feature to grade correlation")
plt.xlabel("Feature names")
plt.ylabel("Correlation to grade")
plt.xticks(ordinal_values-0.5, feature_names[top_ten_indices_lv])

for i, v in enumerate(pearson_correlation_lv[top_ten_indices_lv]):
    plt.text(ordinal_values[i] - 0.2, v + 0.001, str(round(v, 4)), rotation=45)

#plt.figure(figsize=(15, 10))
plt.bar(ordinal_values, pearson_correlation_ls[top_ten_indices_ls], color="green", align="center", label="Lexical Sophistication", width=0.2)
plt.title("Feature to grade correlation")
plt.xlabel("Feature names")
plt.ylabel("Correlation to grade")
plt.xticks(ordinal_values, zip(feature_names[top_ten_indices_lv], feature_names[top_ten_indices_ls+14]), rotation=45)

for i, v in enumerate(pearson_correlation_ls[top_ten_indices_ls]):
    plt.text(ordinal_values[i], v + 0.001, str(round(v, 4)), rotation=45)

plt.legend()
plt.show()



highest_value_index_lv = np.argmax(pearson_correlation_lv)

plt.figure(figsize=(15, 10))
plt.plot(feature_matrix[:, highest_value_index_lv], grades, "*")
plt.gray()
plt.title("Highest correlation feature with the grades("+ feature_names[highest_value_index_lv] + ")")
plt.xlabel(feature_names[highest_value_index_lv])
plt.ylabel("Grades")
plt.show()


highest_value_index_ls = np.argmax(pearson_correlation_ls)

plt.figure(figsize=(15, 10))
plt.plot(feature_matrix[:, highest_value_index_ls+14], grades, "*")
plt.gray()
plt.title("Highest correlation feature with the grades("+ feature_names[highest_value_index_ls + 14] + ")")
plt.xlabel(feature_names[highest_value_index_ls+14])
plt.ylabel("Grades")
plt.show()

<h2>Level C1: Lexical Sophistication feature table</h2>

In [ ]:
pearson_correlation_ls = np.array(pearson_correlation_ls)
p_scores_ls = np.array(p_scores_ls)

data = np.stack([feature_names[14:], pearson_correlation_ls, p_scores_ls])

df = pd.DataFrame(data.T, index=np.arange(14), columns=["Feature", "Pearson_Correlation", "p_score"])
df